In [1]:
import pandas as pd
import requests
from shapely import wkt
from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt
import contextily as cx
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import os
import rtree

In [4]:
gdf = gpd.read_file("work/data/stoej.json")
gdf_edges = gpd.read_file("work/data/edges.json")


#casting to new epsg in order to use meters in the buffer zone
gdf = gdf.to_crs(epsg=25832)
gdf_edges = gdf_edges.to_crs(epsg=25832)



In [ ]:
def find_top_matches(polygon, lines_df, buffer_dist=15, max_matches=3):
    buffered_poly = polygon.buffer(buffer_dist) 

    # Potential matches within the buffered area
    potential_matches_idx = list(lines_index.intersection(buffered_poly.bounds))
    potential_matches = lines_df.iloc[potential_matches_idx].copy()

    if not potential_matches.empty:
        potential_matches['intersect_area'] = potential_matches.apply(lambda row: row['geometry'].intersection(polygon).area, axis=1)
        return potential_matches.sort_values('intersect_area', ascending=False).head(max_matches)
    else:
        return None  

counter = 0

# Build Spatial Index
lines_index = rtree.index.Index()
for idx, row in gdf_edges.iterrows():
    lines_index.insert(idx, row['geometry'].bounds)

results = []

for index, row in gdf.iterrows():  
    polygon = row['geometry']
    top_matches = find_top_matches(polygon, gdf_edges)
    counter += 1

    if top_matches is not None:
        top_matches.loc[:, 'polygon_index'] = index
        isov1_value = row['isov1']
        isov2_value = row['isov2']
        top_matches.loc[:, 'isov1'] = isov1_value 
        top_matches.loc[:, 'isov2'] = isov2_value
        results.append(top_matches)
    if counter == 150:
        break

if results:
    final_gdf = gpd.GeoDataFrame(pd.concat(results, ignore_index=True),crs="EPSG:2643")
